In [1]:
#@title Encoders
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

class Encoder(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """

    def __init__(self, features, feature_dim,
                 embed_dim, adj_lists, aggregator,
                 num_sample=10,
                 base_model=None, gcn=False, cuda=False,
                 kernel="GCN",
                 feature_transform=False):
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
            torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        init.xavier_uniform(self.weight)

        self.kernel = kernel

        num_mlp_layers = 2
        num_layers = 5
        self.num_layers = num_layers

        input_dim = feature_dim
        hidden_dim = 512
        output_dim = embed_dim
        self.mlps = torch.nn.ModuleList()
        # List of batchnorms applied to the output of MLP (input of the final prediction linear layer)
        self.batch_norms = torch.nn.ModuleList()

        for layer in range(self.num_layers-1):
            if layer == 0:
                self.mlps.append(
                    MLP(num_mlp_layers, input_dim, hidden_dim, hidden_dim))
            else:
                self.mlps.append(
                    MLP(num_mlp_layers, hidden_dim, hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
        self.mlps.append(
                    MLP(num_mlp_layers, hidden_dim, hidden_dim, embed_dim))
        self.batch_norms.append(nn.BatchNorm1d(embed_dim))

        # Linear function that maps the hidden representation at dofferemt layers into a prediction score
        self.linears_prediction = torch.nn.ModuleList()
        for layer in range(num_layers):
            if layer == 0:
                self.linears_prediction.append(
                    nn.Linear(input_dim, output_dim))
            else:
                self.linears_prediction.append(
                    nn.Linear(hidden_dim, output_dim))

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.

        nodes     -- list of nodes
        """
        # print('encoder:', nodes)
        if self.kernel == "GIN":
            neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes],
                                                  self.num_sample, average="sum")
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            h = torch.add(self_feats, neigh_feats)
            for layer in range(self.num_layers):
                pooled_rep = self.mlps[layer](h)
                h = self.batch_norms[layer](pooled_rep)
                h = F.relu(h)
            combined = h.t()
        else:
            neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes],
                                                  self.num_sample)
            if not self.gcn:
                if self.cuda:
                    self_feats = self.features(torch.LongTensor(nodes).cuda())
                else:
                    self_feats = self.features(torch.LongTensor(nodes))
                combined = torch.cat([self_feats, neigh_feats], dim=1)
            else:
                combined = neigh_feats

            if self.kernel == "GAT":
                combined = F.elu(self.weight.mm(combined.t()))
            else:
                combined = F.relu(self.weight.mm(combined.t()))
        return combined


In [2]:
#@title Aggregators
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

import random

from torch.nn import init

"""
Set of modules for aggregating embeddings of neighbors.
"""


class MeanAggregator(nn.Module):
    """
    Aggregates a node's embeddings using mean of neighbors' embeddings
    """

    def __init__(self, features, features_dim=4096, cuda=False, gcn=False, kernel="GCN"):
        """
        Initializes the aggregator for a specific graph.

        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(MeanAggregator, self).__init__()

        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        self.softmax = nn.Softmax(dim=1)

        self.kernel = kernel
        self.attention = True if kernel == "GAT" else "False"

        self.in_features = features_dim
        self.out_features = features_dim

        self.weight = nn.Parameter(
            torch.FloatTensor(self.in_features, self.out_features, ))
        self.a = nn.Parameter(
            torch.FloatTensor(2 * self.out_features, 1))
        init.xavier_uniform(self.weight)
        init.xavier_uniform(self.a)

        self.alpha = 0.2
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, nodes, to_neighs, num_sample=10, average="mean"):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh,
                                        num_sample,
                                        )) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs
        if self.gcn:
            samp_neighs = [samp_neigh | set([nodes[i]])
                           for i, samp_neigh in enumerate(samp_neighs)]

        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n: i for i, n in enumerate(unique_nodes_list)}

        # print('agg:', nodes)
        # print('agg unique:', unique_nodes_list)

        column_indices = [unique_nodes[n]
                          for samp_neigh in samp_neighs for n in samp_neigh]
        row_indices = [i for i in range(len(samp_neighs))
                       for j in range(len(samp_neighs[i]))]

        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        mask[row_indices, column_indices] = 1

        attention_mask = Variable(torch.full(
            (len(samp_neighs), len(unique_nodes)), np.inf))
        attention_mask[row_indices, column_indices] = 0

        zero_vec = -9e15 * torch.ones_like(mask)

        if self.cuda:
            mask = mask.cuda()
            attention_mask.cuda()

        num_neigh = mask.sum(1, keepdim=True)
        for ni, num in enumerate(num_neigh):
            if num == 0:
                num_neigh[ni] = 1

        if self.cuda:
            embed_matrix = self.features(
                torch.LongTensor(unique_nodes_list).cuda())
            feature_matrix = self.features(torch.LongTensor(nodes).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
            feature_matrix = self.features(torch.LongTensor(nodes))

        if self.kernel == "GAT":
            # attention_matrix = feature_matrix.mm(embed_matrix.t())
            #             # attention_matrix = attention_matrix.mul(mask)
            #             # attention_matrix = attention_matrix - attention_mask
            #             # attention = self.softmax(attention_matrix)
            # mask = mask.mul(attention)
            feature_matrix = torch.mm(feature_matrix, self.weight)
            embed_matrix = torch.mm(embed_matrix, self.weight)
            N = feature_matrix.size()[0]
            M = embed_matrix.size()[0]

            a_input = torch.cat([feature_matrix.repeat(1, M).view(N * M, -1), embed_matrix.repeat(N, 1)],
                                dim=1).view(N, -1, 2 * self.out_features)
            attention_matrix = self.leakyrelu(
                torch.matmul(a_input, self.a).squeeze(2))
            # print(attention_matrix.size())
            # print(mask.size())

            # attention_matrix = feature_matrix.mm(embed_matrix.t())

            attention = torch.where(mask > 0, attention_matrix, zero_vec)
            attention = self.softmax(attention)
            # mask = mask.mul(attention)
            to_feats = attention.mm(embed_matrix)
        elif self.kernel == "GCN":
            if average == "mean":
                mask = mask.div(num_neigh)
            to_feats = mask.mm(embed_matrix)
        elif self.kernel == "GIN":
            to_feats = mask.mm(embed_matrix)

        return to_feats


class AttentionAggregator(nn.Module):
    """
    Aggregates a node's embeddings with attention
    """

    def __init__(self, features, in_features=4096, out_features=1024, cuda=False, gcn=False, attention_dim=512):
        """
        Initializes the aggregator for a specific graph.

        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(AttentionAggregator, self).__init__()

        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        self.softmax = nn.Softmax(dim=1)
        self.attention = True

        self.in_features = in_features
        self.out_features = out_features

        self.weight = nn.Parameter(
            torch.FloatTensor(out_features, in_features))

        self.a = nn.Parameter(
            torch.FloatTensor(2 * out_features, 1))

    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh,
                                        num_sample,
                                        )) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs
        if self.gcn:
            samp_neighs = [samp_neigh | set([nodes[i]])
                           for i, samp_neigh in enumerate(samp_neighs)]

        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n: i for i, n in enumerate(unique_nodes_list)}

        column_indices = [unique_nodes[n]
                          for samp_neigh in samp_neighs for n in samp_neigh]
        row_indices = [i for i in range(len(samp_neighs))
                       for j in range(len(samp_neighs[i]))]

        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        mask[row_indices, column_indices] = 1

        zero_vec = -9e15 * torch.ones_like(mask)

        if self.cuda:
            mask = mask.cuda()

        num_neigh = mask.sum(1, keepdim=True)
        for ni, num in enumerate(num_neigh):
            if num == 0:
                num_neigh[ni] = 1

        if self.cuda:
            embed_matrix = self.features(
                torch.LongTensor(unique_nodes_list).cuda())
            feature_matrix = self.features(torch.LongTensor(nodes).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
            feature_matrix = self.features(torch.LongTensor(nodes))

        attention_matrix = feature_matrix.mm(embed_matrix.t())
        attention = torch.where(mask > 0, attention_matrix, zero_vec)
        attention = self.softmax(attention)

        to_feats = attention.mm(embed_matrix)
        return to_feats


In [3]:
#@title MLP
import torch
import torch.nn as nn
import torch.nn.functional as F

###MLP with lienar output
class MLP(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        '''
            num_layers: number of layers in the neural networks (EXCLUDING the input layer). If num_layers=1, this reduces to linear model.
            input_dim: dimensionality of input features
            hidden_dim: dimensionality of hidden units at ALL layers
            output_dim: number of classes for prediction
            device: which device to use
        '''
    
        super(MLP, self).__init__()

        self.linear_or_not = True #default is linear model
        self.num_layers = num_layers

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            #Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            #Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()
        
            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            for layer in range(num_layers - 1):
                self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            #If linear model
            return self.linear(x)
        else:
            #If MLP
            h = x
            for layer in range(self.num_layers - 1):
                h = F.relu(self.batch_norms[layer](self.linears[layer](h)))
            return self.linears[self.num_layers - 1](h)



In [4]:
#@title Model Multi
import functools

import sklearn
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable

import numpy as np
import time
import random

import warnings
import sklearn.exceptions


warnings.filterwarnings(
    "ignore", category=sklearn.exceptions.UndefinedMetricWarning)


def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        start_time = time.time()
        res = func(*args, **kw)
        end_time = time.time()
        print('%s executed in %ss' % (func.__name__, end_time - start_time))
        return res

    return wrapper


class DiseasesClassifier(nn.Module):

    def __init__(self, num_classes, enc):
        super(DiseasesClassifier, self).__init__()
        self.enc = enc

        # self.xent = nn.CrossEntropyLoss()
        self.xent = nn.BCEWithLogitsLoss()

        self.weight = nn.Parameter(
            torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

        self.a = nn.Linear(enc.embed_dim, 1)

    @staticmethod
    def binary_loss(y_pred, y):
        y_pred = torch.from_numpy(y_pred)
        logits = (y * y_pred.clamp(1e-12).log() + (1 - y)
                  * (1 - y_pred).clamp(1e-12).log()).mean()
        return -logits

    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()

    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        return self.xent(scores, labels)

    def forward_hinge(self, nodes):
        embeds = self.enc(nodes)
        return self.a(embeds.t())  # a = nn.Linear(enc.embed_dim, 1)

    def hinge_loss(self, nodes, labels):
        h_loss = self.forward_hinge(nodes)
        return torch.mean(torch.clamp(1 - h_loss * labels, min=0))


def evaluate(data_name, val_output, test_labels, val, topk=(1, 2, 3, 4, 5,)):
    print("----" * 25)
    print()
    print("%s: " % data_name)

    # shape: batchnum * classnum
    target = torch.LongTensor(test_labels[val])
    output = val_output  # shape: batchnum * classnum

    # print(target.shape, output.shape)
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    # print(pred)
    correct = torch.zeros_like(pred)
    for i in range(batch_size):
        for k in range(maxk):
            correct[i, k] = 1 if target[i][pred[i, k]] == 1 else 0
    correct = correct.t()

    correct_target = target.sum(1, keepdim=True).squeeze().float()

    for k in topk:
        correct_k = correct[:k].sum(0, keepdim=True).squeeze().float()

        precision_k = 0.0
        # recall_k = 0.0
        for i in range(0, batch_size):
            # _k = k if k < correct_target[i].data else correct_target[i]
            _k = k
            precision_k += correct_k[i] / _k
            # recall_k += correct_k[i] / correct_target[i]
        precision_k = precision_k / batch_size
        # recall_k = recall_k / batch_size

        # print("precision @", k, precision_k.data)
        # print("recall @", k, recall_k.data)

        # precision_k = correct_k / k
        # precision_k = precision_k.sum() / batch_size

        recall_k = correct_k / correct_target
        recall_k = recall_k.sum() / batch_size

        f1_k = 2 * precision_k * recall_k / (precision_k + recall_k)

        print("precision @ %d : %.5f, recall @ %d : %.5f, f1 @ %d : %.5f" % (
            k, precision_k.data, k, recall_k.data, k, f1_k.data))
        # print("precision @", k, precision_k.data)
        # print("recall @", k, recall_k.data)
        # print("f1 @", k, f1_k.data)

        # print("precision@%d: %f" & (k, precision_k))
        # print("recall@%d: %f" & (k, recall_k))
    print()


class DiseasesPredictor:
    def __init__(self, feat_data, b_labels, multi_class_num, labels, adj_lists, feature_dim,
                 train_enc_num, train_enc_dim, train_sample_num,
                 train, test,
                 kernel='gcn',  topk=(1, 2, 3, 4, 5,),
                 weights_flag=False, weights=[0.5, 0.5],
                 gcn=False, agg_gcn=True, cuda=False):

        self.cuda = cuda
        self.gcn = gcn
        self.agg_gcn = agg_gcn

        self.train_original = train.copy()
        self.test_original = test.copy()

        self.train = train
        self.test = test
        self.test_rare = [i for i in np.where(
            (b_labels > 0))[0].squeeze() if i in self.test]
        self.test_rare_index = [self.test.index(i) for i in self.test_rare]

        self.b_labels = b_labels
        self.labels = labels

        self.bi_class_num = 2
        self.multi_class_num = multi_class_num

        # nodes' features (random setting)
        self.features = nn.Embedding(len(feat_data), feature_dim)
        self.features.weight = nn.Parameter(
            torch.FloatTensor(feat_data), requires_grad=False)
        self.adj_lists = adj_lists  # edges' information

        # model parameters
        self.train_enc_dim = train_enc_dim
        self.train_enc_num = train_enc_num
        self.kernel = kernel
        self.attention = True if kernel == "GAT" else False
        self.feature_dim = feature_dim
        self.train_sample_num = train_sample_num

        self.topk = topk

        # weighted cross-entropy
        self.weights_flag = weights_flag
        self.class_weights = torch.FloatTensor(weights)

        # labels for test
        self.test_b_labels = b_labels
        self.test_adj = adj_lists

        # inductive settings
        self.is_inductive = False
        self.test_sample_num = train_sample_num
        self.test_features = self.features

        # build aggregator and encoders
        # default: transductive setting

        self.agg1 = MeanAggregator(self.features, features_dim=feature_dim,
                                   cuda=self.cuda, kernel=self.kernel, gcn=self.agg_gcn)
        self.enc1 = Encoder(self.features, feature_dim, train_enc_dim[0], adj_lists, self.agg1,
                            gcn=self.gcn, cuda=self.cuda, kernel=self.kernel)

        self.agg2 = MeanAggregator(lambda nodes: self.enc1(nodes).t(), features_dim=self.enc1.embed_dim,
                                   cuda=self.cuda, kernel=self.kernel, gcn=self.agg_gcn)
        self.enc2 = Encoder(lambda nodes: self.enc1(nodes).t(), self.enc1.embed_dim, train_enc_dim[1], adj_lists,
                            self.agg2, base_model=self.enc1, gcn=self.gcn, cuda=self.cuda, kernel=self.kernel)

        self.agg3 = MeanAggregator(lambda nodes: self.enc2(nodes).t(), features_dim=self.enc2.embed_dim,
                                   cuda=self.cuda, kernel=self.kernel, gcn=self.agg_gcn)
        self.enc3 = Encoder(lambda nodes: self.enc2(nodes).t(), self.enc2.embed_dim, train_enc_dim[2], adj_lists,
                            self.agg3, base_model=self.enc2, gcn=self.gcn, cuda=self.cuda, kernel=self.kernel)
        self.agg4 = MeanAggregator(lambda nodes: self.enc3(nodes).t(), features_dim=self.enc3.embed_dim,
                                   cuda=self.cuda, kernel=self.kernel, gcn=self.agg_gcn)
        self.enc4 = Encoder(lambda nodes: self.enc3(nodes).t(), self.enc3.embed_dim, train_enc_dim[3], adj_lists,
                            self.agg4, base_model=self.enc3, gcn=self.gcn, cuda=self.cuda, kernel=self.kernel)
        self.enc1.num_samples = self.train_sample_num[0]
        self.enc2.num_samples = self.train_sample_num[1]
        self.enc3.num_samples = self.train_sample_num[2]
        self.enc4.num_samples = self.train_sample_num[3]

    def set_classifier(self, class_num, train_enc_num):
        classifier = DiseasesClassifier(class_num, self.enc2)
        if train_enc_num == 1:
            classifier = DiseasesClassifier(class_num, self.enc1)
        elif train_enc_num == 2:
            classifier = DiseasesClassifier(class_num, self.enc2)
        elif train_enc_num == 3:
            classifier = DiseasesClassifier(class_num, self.enc3)
        elif train_enc_num == 4:
            classifier = DiseasesClassifier(class_num, self.enc4)
        return classifier

    def run(self, loop_num, batch_num, lr):
        if loop_num is None:
            loop_num = [100, 500]

        multi_classifier = self.set_classifier(
            class_num=self.multi_class_num, train_enc_num=self.train_enc_num)

        self.__train__(multi_classifier, train=self.train, labels=self.labels,
                       loop_num=loop_num, batch_num=batch_num, lr=lr)

        multi_result_direct = multi_classifier.forward(self.test)

        evaluate("multi classification (overall)",
                 multi_result_direct,
                 self.labels,
                 self.test,
                 topk=self.topk)
        print("len of rare:", len(self.test_rare_index))
        evaluate("multi classification (rare)",
                 multi_result_direct[self.test_rare_index],
                 self.labels,
                 self.test_rare,
                 topk=self.topk)

    def __train__(self, selected_model, train, labels, loop_num=100, batch_num=512, lr=0.01):
        np.random.seed(1)
        random.seed(1)

        optimizer = torch.optim.SGD(
            filter(lambda p: p.requires_grad, selected_model.parameters()), lr=lr)
        # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, selected_model.parameters()), lr=lr, betas=(0.9, 0.99))
        times = []

        for batch in range(loop_num):
            batch_nodes = train[:batch_num]
            random.shuffle(train)
            start_time = time.time()
            optimizer.zero_grad()
            loss = selected_model.loss(batch_nodes,
                                       Variable(torch.FloatTensor(labels[np.array(batch_nodes, dtype=np.int64)])))
            loss.backward()
            optimizer.step()
            end_time = time.time()
            times.append(end_time - start_time)
            if batch % 100 == 0:
              print(batch, loss.data)

        print()
        print("Average batch time:", np.mean(times))
        print()


In [5]:
#@title Model


In [6]:
#@title Unsupervised


In [7]:
import time
import pickle
import numpy as np

file_path = "/content/drive/MyDrive/GCNModel/sample_data2/sample_graph"
node_list = pickle.load(open(file_path + ".nodes.pkl", "rb"))
adj_lists = pickle.load(open(file_path + ".adj.pkl", "rb"))
rare_patient = pickle.load(open(file_path + ".rare.label.pkl", "rb"))
labels = pickle.load(open(file_path + ".label.pkl", "rb"))
node_map = pickle.load(open(file_path + ".map.pkl", "rb"))
train = pickle.load(open(file_path + ".train.pkl", "rb"))
test = pickle.load(open(file_path + ".test.pkl", "rb"))

multi_class_num = 101
feature_dim = 10000
epoch = 8000
batch_num = 200
lr = 0.3
feat_data = np.random.random((50000, feature_dim))
train_enc_dim = [1000, 1000, 1000, 1000]
t1 = time.time()
model = DiseasesPredictor(feat_data=feat_data,
                          b_labels=rare_patient,
                          multi_class_num=multi_class_num,
                          labels=labels,
                          adj_lists=adj_lists,
                          feature_dim=feature_dim,
                          train_enc_num=1,
                          train_enc_dim=train_enc_dim,
                          train_sample_num=[5, 5, 5, 5],
                          train=train, test=test,
                          kernel='GCN',
                          topk=(1, 2, 3, 4, 5,))

model.run(epoch, batch_num, lr)  # epoch, batch_num, lr
print(feature_dim, train_enc_dim)
print("running time:", time.time()-t1, "s")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


0 tensor(0.7340)
100 tensor(0.0485)
200 tensor(0.0412)
300 tensor(0.0342)
400 tensor(0.0271)
500 tensor(0.0214)
600 tensor(0.0167)
700 tensor(0.0130)
800 tensor(0.0105)
900 tensor(0.0086)
1000 tensor(0.0072)
1100 tensor(0.0062)
1200 tensor(0.0053)
1300 tensor(0.0047)
1400 tensor(0.0041)
1500 tensor(0.0037)
1600 tensor(0.0034)
1700 tensor(0.0030)
1800 tensor(0.0028)
1900 tensor(0.0026)
2000 tensor(0.0024)
2100 tensor(0.0022)
2200 tensor(0.0021)
2300 tensor(0.0019)
2400 tensor(0.0018)
2500 tensor(0.0017)
2600 tensor(0.0016)
2700 tensor(0.0015)
2800 tensor(0.0015)
2900 tensor(0.0014)
3000 tensor(0.0013)
3100 tensor(0.0013)
3200 tensor(0.0012)
3300 tensor(0.0012)
3400 tensor(0.0011)
3500 tensor(0.0011)
3600 tensor(0.0010)
3700 tensor(0.0010)
3800 tensor(0.0010)
3900 tensor(0.0009)
4000 tensor(0.0009)
4100 tensor(0.0009)
4200 tensor(0.0009)
4300 tensor(0.0008)
4400 tensor(0.0008)
4500 tensor(0.0008)
4600 tensor(0.0008)
4700 tensor(0.0007)
4800 tensor(0.0007)
4900 tensor(0.0007)
5000 tensor(